In [12]:
#please make sure you have the package: pip install nameparser
#current numpy version is 1.21
import spacy
import os
from spacy.tokens import Span
from datetime import datetime
from nameparser.parser import HumanName
import re
from nltk.corpus import stopwords
import json
import glob
import en_core_web_sm
from collections import Counter

class NLP:
    def __init__(self,write=True):
        self.w=write
        
    def get_article_titles(self):
        file_list = glob.glob("*.txt") 
        fnames = []
        for fname in file_list:
            fname = fname.split('_')[0]
            fnames.append(fname)
        with open (\'article_titles_Spacy.json', 'w') as output_title:
            output_title.write(json.dumps(fnames))
    
    def get_researcher_names(self):
        person_articles = []
        nlp = en_core_web_sm.load()
        for article in glob.glob('./*.txt'):
            article = open(article, 'r',encoding='utf-8')
            # read all text
            text = article.read()
            t000=datetime.now()
        
            #new_ents = []
            #for ent in text.ents:
                #if ent.label_ == "PERSON":
                    #new_ent = Span(text, ent.start, ent.end, label=ent.label)
                    #person_articles.append(new_ent)
            text = nlp(text)
            person = [ent.text for ent in text.ents if ent.label_=='PERSON']
            person_articles.append(person)
            
        with open ('researcher_names_articles_Spacy.json', 'w') as output_name:
            output_name.write(json.dumps(person_articles))
                
                
        t001=datetime.now()
        dt00=t001-t000
        print('Name Extraction is completed after '+str(dt00.total_seconds())+' seconds')

In [13]:
spacy=NLP(write=True)
spacy.get_article_titles()
spacy.get_researcher_names()
#nlp.dictionary_titles_names()

Name Extraction is completed after 0.098022 seconds


In [15]:
article_titles = []

with open('article_titles_Spacy.json') as output_title:
    for title in output_title:
        article_titles.append(json.loads(title))

researcher_names = []
        
with open('researcher_names_articles_Spacy.json') as output_name:
    for name in output_name:
        researcher_names.append(json.loads(name))
                
genders = []
if os.path.exists('researcher_gender_articles_Spacy.json'):
    with open('researcher_gender_articles_Spacy.json') as output_gender:
        genders = json.load(output_gender)

researcher_dict = {}
for i in range(len(researcher_names[0])):
    researcher_dict[article_titles[0][i]] = researcher_names[0][i]

                
def get_next_researcher():
    return list(researcher_dict.values())[len(genders)]


get_next_researcher()

['Stephen Fried',
 'Anneliese M. Faustino',
 'Mikhail Makarov',
 'Alma C. Sanchez Rocha',
 'Ivan Cherepashuk',
 'Robin Krystufek',
 'Klara Hlouchova',
 'Volha Dzmitruk',
 'Tatsiana Charnavets',
 'Michal Lebl',
 'Kosuke Fujishima']

In [16]:
%%javascript
        
function set_gender(gender){
    var kernel = IPython.notebook.kernel;
    kernel.execute("genders.append(" + gender + ")");
    load_next_researcher();
}

function handle_output(out){
    var res = out.content.data["text/plain"];
    $("div#researcher").html(res);
}
        
function load_next_researcher(){
    var code_input = "get_next_researcher()";
    var kernel = IPython.notebook.kernel;
    var callbacks = { 'iopub' : {'output' : handle_output}};
    kernel.execute(code_input, callbacks, {silent:false});
}

<IPython.core.display.Javascript object>

In [17]:
%%html
<div name="researcherbox">
    Instructions: Click in textbox. Enter a 1 if the researcher is female, enter 0 otherwise. <br>
Researcher Name: <div id="researcher" value="text"></div><br>
<input type=researcher_names id="capture"></input><br>
</div>
        
<script>

function set_gender(gender){
    var kernel = IPython.notebook.kernel;
    kernel.execute("genders.append(" + gender + ")");
    load_next_researcher();
}

function handle_output(out){
    var res = out.content.data["text/plain"];
    $("div#researcher").html(res);
}
        
function load_next_researcher(){
    var code_input = "get_next_researcher()";
    var kernel = IPython.notebook.kernel;
    var callbacks = { 'iopub' : {'output' : handle_output}};
    kernel.execute(code_input, callbacks, {silent:false});
}

$("input#capture").keypress(function(e) {
if(e.which == 48) {
    set_gender(0);
    $("input#capture").val("");
}else if (e.which == 49){
    set_gender(1);
    $("input#capture").val("");
  }
});
        
load_next_researcher();
</script>

In [18]:
with open('researcher_genders.json', 'w') as output_gender:
    json.dump(genders,output_gender)